# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

#import text cleasing libraries
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#import text preprocessing and modeling libraries
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Project3.db')
df = pd.read_sql("SELECT * FROM df", engine)
X = df['message']
Y = df.iloc[:, 4:]

In [15]:
X.shape

(26216,)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    return [i for i in ''.join([c for c in text if c not in string.punctuation]).split()
            if i.lower() not in stopwords.words('english')]


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(MultinomialNB()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

pipeline.fit(X_train,Y_train)
predictions = pipeline.predict(X_test)


/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
for i in range(0, len(Y_test.columns)):
    print(confusion_matrix(Y_test.values[:,i], predictions[:,i]),
          '\n',
          classification_report(Y_test.values[:,i], predictions[:,i]))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[ 131 1691    0]
 [  33 5953    0]
 [   6   51    0]] 
              precision    recall  f1-score   support

          0       0.77      0.07      0.13      1822
          1       0.77      0.99      0.87      5986
          2       0.00      0.00      0.00        57

avg / total       0.77      0.77      0.69      7865

[[6508   45]
 [1080  232]] 
              precision    recall  f1-score   support

          0       0.86      0.99      0.92      6553
          1       0.84      0.18      0.29      1312

avg / total       0.85      0.86      0.82      7865

[[7831    0]
 [  34    0]] 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7831
          1       0.00      0.00      0.00        34

avg / total       0.99      1.00      0.99      7865

[[3869  774]
 [1200 2022]] 
              precision    recall  f1-score   support

          0       0.76      0.83      0.80      4643
          1       0.72      0.63      0.67      322

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = { 'classifier__estimator__alpha': [0.2, 1.0]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train,Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] classifier__estimator__alpha=0.2 ................................


/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


[CV]  classifier__estimator__alpha=0.2, score=0.19306849762955697, total= 1.3min
[CV] classifier__estimator__alpha=0.2 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


[CV]  classifier__estimator__alpha=0.2, score=0.17851888180480627, total= 1.3min
[CV] classifier__estimator__alpha=0.2 ................................


...

[CV]  classifier__estimator__alpha=1.0, score=0.15546836684649337, total= 1.2min
[CV] classifier__estimator__alpha=1.0 ................................


/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


[CV]  classifier__estimator__alpha=1.0, score=0.14713094654242276, total= 1.2min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 12.2min finished
/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function tokenize at 0x7fa05936b0d0>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'classifier__estimator__alpha': [0.2, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [11]:
# Check out the best number of estimators the machine decided for us
cv.best_params_

{'classifier__estimator__alpha': 0.2}

In [12]:
predictions = cv.predict(X_test)
for i in range(0, len(Y_test.columns)):
    print(confusion_matrix(Y_test.values[:,i], predictions[:,i]),
          '\n',
          classification_report(Y_test.values[:,i], predictions[:,i]))

[[ 510 1312    0]
 [ 187 5795    4]
 [   7   38   12]] 
              precision    recall  f1-score   support

          0       0.72      0.28      0.40      1822
          1       0.81      0.97      0.88      5986
          2       0.75      0.21      0.33        57

avg / total       0.79      0.80      0.77      7865

[[6320  233]
 [ 663  649]] 
              precision    recall  f1-score   support

          0       0.91      0.96      0.93      6553
          1       0.74      0.49      0.59      1312

avg / total       0.88      0.89      0.88      7865

[[7831    0]
 [  34    0]] 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7831
          1       0.00      0.00      0.00        34

avg / total       0.99      1.00      0.99      7865

[[3628 1015]
 [1029 2193]] 
              precision    recall  f1-score   support

          0       0.78      0.78      0.78      4643
          1       0.68      0.68      0.68      322

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[7342    6]
 [ 517    0]] 
              precision    recall  f1-score   support

          0       0.93      1.00      0.97      7348
          1       0.00      0.00      0.00       517

avg / total       0.87      0.93      0.90      7865

[[7505    6]
 [ 350    4]] 
              precision    recall  f1-score   support

          0       0.96      1.00      0.98      7511
          1       0.40      0.01      0.02       354

avg / total       0.93      0.95      0.93      7865

[[7457    4]
 [ 399    5]] 
              precision    recall  f1-score   support

          0       0.95      1.00      0.97      7461
          1       0.56      0.01      0.02       404

avg / total       0.93      0.95      0.92      7865

[[7704    2]
 [ 159    0]] 
              precision    recall  f1-score   support

          0       0.98      1.00      0.99      7706
          1       0.00      0.00      0.00       159

avg / total       0.96      0.98      0.97      7865

[[7827    1]
 [  37    0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
pickle.dump(cv,open('classifier.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.